# Functions used to compute the different scores

They can be moved to a util.py file later.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
import pyarrow.feather as feather
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# function for utils later to get the average weights of articles from a DataFrame containing path information

def calculate_avg_article_weights(df, count_cutoff=30, scaling='standard'):
    """
    Calculate the average weights of articles from a DataFrame containing path information.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
            - 'simplified_path_length': Length of the simplified path
            - 'distance': Distance associated with the path
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.
        count_cutoff (int): Minimum number of appearances for an article to be considered

    Returns:
        pd.DataFrame: A DataFrame containing:
            - 'article': Article name
            - 'n_appearances': Number of times the article appeared in paths
            - 'weighted_avg': Weighted average of distances for the article
    """
    # Copy and preprocess the DataFrame
    df = df[['simplified_path', 'simplified_path_length', 'distance']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Calculate weight for each path
    df['weight'] = df['distance'] / df['simplified_path_length']

    # Initialize an empty DataFrame to store results
    avg_article_weight_df = pd.DataFrame(columns=['article', 'n_appearances', 'weighted_avg'])
    avg_article_weight_df.set_index('article', inplace=True)

    # Iterate through each row to calculate weights
    for _, row in df.iterrows():
        weight = row['weight']
        simplified_path = row['simplified_path']

        for article in simplified_path:
            if article not in avg_article_weight_df.index:
                avg_article_weight_df.loc[article] = [0, 0]

            # Update counts and weighted sums
            avg_article_weight_df.at[article, 'n_appearances'] += 1
            avg_article_weight_df.at[article, 'weighted_avg'] += weight

    # Calculate the weighted average by dividing weighted sum by counts
    avg_article_weight_df['weighted_avg'] = avg_article_weight_df['weighted_avg'] / avg_article_weight_df['n_appearances']

    # Filter out articles that appear less than the cutoff
    avg_article_weight_df = avg_article_weight_df[avg_article_weight_df['n_appearances'] >= count_cutoff]

    # Normalize the weighted average
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()

    avg_article_weight_df[scaling] = scaler.fit_transform(avg_article_weight_df[['weighted_avg']])


    print(f"Number of unique articles after weighting: {avg_article_weight_df.shape[0]}")

    return avg_article_weight_df#.reset_index()


# ------------------------------------------------


# code a function that returns the ratio of the number of times an article appears in unfinished paths over the total number of times it appears

def ratio_unfinished(in_df, count_cutoff=30, scaling='standard'):
    """
    Calculate the ratio of the number of times an article appears in unfinished paths over the total number of times it appears.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
        count_cutoff (int): Minimum number of appearances for an article to be considered
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.Series: A Series containing the ratio for each article
    """
    # Copy and preprocess the DataFrame
    df = in_df[['simplified_path', 'finished']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize a dictionary to store counts
    article_counts = {}
    unfinished_counts = {}

    # Iterate through each row to calculate counts
    for _, row in df.iterrows():
        simplified_path = row['simplified_path']
        finished = row['finished']

        for article in simplified_path:
            article_counts[article] = article_counts.get(article, 0) + 1
        
        if not finished:
            for article in simplified_path:
                unfinished_counts[article] = unfinished_counts.get(article, 0) + 1

    # Convert the dictionary to a Series
    article_counts = pd.Series(article_counts)
    unfinished_counts = pd.Series(unfinished_counts)

    ratio = unfinished_counts / article_counts

    ratio_df = pd.DataFrame({
    'n_appearances': article_counts,
    'unfinished_counts': unfinished_counts,
    'unfinished_ratio': ratio
    }).fillna(0)

    # cut off
    ratio_df = ratio_df[ratio_df['n_appearances'] >= count_cutoff]

    # scaling
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()
    
    ratio_df[scaling] = scaler.fit_transform(ratio_df[['unfinished_ratio']])

    #print(f"Number of unique articles: {len(article_counts)}")
    print(f"Ratio of unfinished over finished paths: {1-df['finished'].mean()}")
    return ratio_df


# ------------------------------------------------


    # code a function that counts the number of dead ends an article has (difference between full path list content and simplified path list content)

def calculate_detour_ratios(in_df, count_cutoff=1, scaling='standard'):
    """
    Calculate the detour ratio for articles based on the full path and simplified path.

    Parameters:
        in_df (pd.DataFrame): Input DataFrame with the following columns:
            - 'full_path': List of articles in the full path
            - 'simplified_path': List of articles in the simplified path
        count_cutoff (int): Minimum number of detours for an article to be considered.
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.DataFrame: A DataFrame containing the detour ratio and scaled values for each article.
    """
    # Copy and preprocess the DataFrame
    df = in_df[['full_path', 'simplified_path']].copy()
    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles
    df['full_path'] = df['full_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize dictionaries to store counts
    detour_counts = {}
    total_counts = {}

    # Iterate through each row to calculate detour counts and total appearances
    for _, row in df.iterrows():
        full_path = row['full_path']
        simplified_path = row['simplified_path']

        # Count total appearances for articles in the full path
        for article in full_path:
            total_counts[article] = total_counts.get(article, 0) + 1

        # Find detour articles by subtracting the simplified path from the full path
        detour_articles = set(full_path) - set(simplified_path)
        for article in detour_articles:
            detour_counts[article] = detour_counts.get(article, 0) + 1

    # Convert counts to Series
    detour_counts = pd.Series(detour_counts)
    total_counts = pd.Series(total_counts)

    # Fill missing detour counts with 0 for articles with no detours
    detour_counts = detour_counts.reindex(total_counts.index, fill_value=0)

    # Calculate detour ratio
    detour_ratios = detour_counts / total_counts

    # Create a DataFrame with detour counts and ratios
    detour_df = pd.DataFrame({
        'detour_count': detour_counts,
        'total_count': total_counts,
        'detour_ratio': detour_ratios
    }).loc[detour_ratios.index]

    # Filter out articles with detour ratio less than the count_cutoff
    detour_df = detour_df[detour_df['total_count'] >= count_cutoff]

    # Normalize the detour ratios
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()

    detour_df[scaling] = scaler.fit_transform(detour_df[['detour_ratio']])

    print(f"Number of unique articles after detour ratio calculation: {len(detour_df)}")

    return detour_df



# ------------------------------------------------

def calc_avg_article_time(df, count_cutoff=30, scaling='standard'):
    """
    Calculate the average speed of articles from a DataFrame containing path information.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'simplified_path': List of articles in the path
            - 'durationInSec': Duration associated with the path
        count_cutoff (int): Minimum number of appearances for an article to be considered
        scaling (str): Type of scaling to use. Options are 'minmax', 'standard', and 'robust'.

    Returns:
        pd.DataFrame: A DataFrame containing:
            - 'article': Article name
            - 'n_appearances': Number of times the article appeared in paths
            - 'avg_speed': Average speed of the article
    """
    # Copy and preprocess the DataFrame
    df = df[['simplified_path', 'durationInSec']].copy()

    df['simplified_path'] = df['simplified_path'].apply(lambda l: l[1:-1])  # Remove start and end articles

    # Initialize an empty DataFrame to store results
    avg_article_speed_df = pd.DataFrame(columns=['article', 'n_appearances', 'avg_speed'])
    avg_article_speed_df.set_index('article', inplace=True)

    # Iterate through each row to calculate speeds
    for _, row in df.iterrows():
        speed = row['durationInSec']
        simplified_path = row['simplified_path']

        for article in simplified_path:
            if article not in avg_article_speed_df.index:
                avg_article_speed_df.loc[article] = [0, 0]

            # Update counts and sums
            avg_article_speed_df.at[article, 'n_appearances'] += 1
            avg_article_speed_df.at[article, 'avg_speed'] += speed

    # Calculate the average speed by dividing sum by counts
    avg_article_speed_df['avg_speed'] = avg_article_speed_df['avg_speed'] / avg_article_speed_df['n_appearances']

    # Filter out articles that appear less than the cutoff
    avg_article_speed_df = avg_article_speed_df[avg_article_speed_df['n_appearances'] >= count_cutoff]

    # Normalize the average speed
    if scaling == 'minmax':
        scaler = MinMaxScaler()
    elif scaling == 'standard':
        scaler = StandardScaler()
    elif scaling == 'robust':
        scaler = RobustScaler()
    
    avg_article_speed_df[scaling] = scaler.fit_transform(avg_article_speed_df[['avg_speed']])

    print(f"Number of unique articles after time calc: {avg_article_speed_df.shape[0]}")

    return avg_article_speed_df#.reset_index()


# COMMENT: could consider really computing the speed instead of the duration. speed = distance / time and then sum up and average.

### And a function for data filtering based on time aspect

In [4]:
def filter_duration(df):
    """
    Filter the DataFrame based on the distance and duration bounds using the IQR method. And downsample to one IpAdress per identifier.

    Parameters:
        df (pd.DataFrame): Input DataFrame with the following columns:
            - 'distance': Distance associated with the path
            - 'durationInSec': Duration associated with the path

    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    filtered_dfs = []  # List to hold filtered data for each distance group

    for d in range(1, int(df['distance'].max()) + 1):
        # Filter the DataFrame for the current distance group
        df_d = df[df['distance'] == d]

        # Compute IQR for 'durationInSec'
        Q1 = df_d['durationInSec'].quantile(0.25)
        Q3 = df_d['durationInSec'].quantile(0.75)
        IQR = Q3 - Q1

        # Calculate upper bound based on IQR
        upper_bound = Q3 + 1.5 * IQR

        # Keep only rows within the upper bound
        filtered_df_d = df_d[df_d['durationInSec'] <= upper_bound]

        # Append filtered group to the list
        filtered_dfs.append(filtered_df_d)

    # Concatenate all filtered groups
    filtered_df = pd.concat(filtered_dfs, ignore_index=True)
    
    # downsample data to one IpAdress per identifier
    downsampled_df = filtered_df.groupby(['hashedIpAddress', 'identifier']).sample(n=1, random_state=42)

    # Calculate the number of removed rows
    removed = df.shape[0] - downsampled_df.shape[0]

    # Print the result
    print(f"In sampling a total of {removed} samples were removed, "
        f"which represents {removed / df.shape[0] * 100:.3f}% of the original data.",
        f"{df.shape[0]} samples remain.")

    return downsampled_df

## Make a composite df with all the different scores

In [5]:
filtered_paths = feather.read_feather('Data/dataframes/filtered_paths.feather')

In [6]:
finished_paths = filtered_paths[filtered_paths['finished']]

# downsample data to one IpAdress per identifier
# this way players can't just learn paths and then play them as fast as possible
finished_paths = finished_paths.groupby(['hashedIpAddress', 'identifier']).sample(n=1, random_state=42)

weight_df = calculate_avg_article_weights(finished_paths, count_cutoff=30, scaling='standard')
time_df = calc_avg_article_time(filter_duration(finished_paths), count_cutoff=30, scaling='standard')
unfinished_atio_df = ratio_unfinished(filtered_paths, count_cutoff=30, scaling='standard')
detour_ratio_df = calculate_detour_ratios(finished_paths, count_cutoff=30, scaling='standard')

/var/folders/sj/219f9qbn3y15yynshcyrrp3c0000gn/T/ipykernel_21684/2940657772.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  avg_article_weight_df.at[article, 'weighted_avg'] += weight


Number of unique articles after weighting: 820
In sampling a total of 2471 samples were removed, which represents 5.437% of the original data. 45451 samples remain.
Number of unique articles after time calc: 776
Ratio of unfinished over finished paths: 0.1762317738926128
Number of unique articles after detour ratio calculation: 871


In [7]:
# Combine the metrics into a composite score
composite_df = pd.DataFrame(index=weight_df.index)
composite_df['weighted_avg'] = weight_df['standard']
composite_df['avg_speed'] = time_df['standard']
composite_df['unfinished_ratio'] = unfinished_atio_df['standard']
composite_df['detour_ratio'] = detour_ratio_df['standard']

composite_df

# | article | weighted_avg | avg_speed | unfinished_ratio | detour_ratio |
# |---------|bigger better |small better|   small better  | small better |

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Philosophy,0.924846,-0.245149,-0.213956,-0.146164
Mathematics,0.893903,-0.200423,-0.896721,-1.029241
Arithmetic,1.100646,0.575692,-0.953803,0.221156
North_Africa,-0.605665,0.060291,0.090040,-0.155223
Africa,0.907139,-0.828117,-0.529661,-0.973283
...,...,...,...,...
United_States_Senate,-1.257078,-1.575148,1.073610,2.311367
Cheese,2.737321,NaN,-0.492802,-0.644868
Nobel_Peace_Prize,0.919959,-0.610105,-0.569058,-0.193466


In [13]:
# rank by highest wieght (remember weight for an article is the average of (distance / simplified_path_length) over all the paths it appears in)
composite_df.sort_values(by='weighted_avg', ascending=False)

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Achilles,4.811790,-2.045308,-0.850784,-1.312944
J._K._Rowling,4.044191,-1.277557,-1.006611,-0.949604
Mario,3.390951,0.830807,-0.859010,-0.277426
Harry_Potter,3.188972,-0.005016,-0.565727,-1.312944
Lead,3.156074,NaN,-0.492802,3.047129
...,...,...,...,...
Anatomy,-2.276821,0.853373,2.486595,0.569815
Irrigation,-2.391556,-0.076705,1.989156,0.646143
Gas,-2.539718,-0.609033,0.473747,3.601376


In [14]:
# sort by speed (so far speed is just avg path time over all the paths an article appears in)
composite_df.sort_values(by='avg_speed')

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
North_Korea,0.431010,-3.003798,0.153795,0.890285
Old_English_language,0.054290,-2.821436,1.878974,0.487956
Korea,1.232629,-2.687098,-0.748606,-0.842254
Suez_Canal,0.080033,-2.453527,-0.979452,1.223017
President_of_the_United_States,1.664244,-2.117642,-1.205954,-0.936918
...,...,...,...,...
Welding,1.771787,NaN,-0.542058,-1.312944
List_of_rivers_by_length,1.915145,NaN,-1.006611,-0.018547
Oxford,-0.075141,NaN,0.456928,1.957111


In [ ]:
# sort by unfinished ratio (ratio of the number of times an article appears in unfinished paths over the total number of times it appears)
composite_df.sort_values(by='unfinished_ratio')

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Australian_Green_Tree_Frog,1.008014,0.458453,-2.364533,-0.795185
Frog,1.556368,NaN,-2.364533,-0.644868
List_of_countries,0.528985,-0.741666,-2.096106,-1.312944
Periodic_table,1.216158,-0.671913,-2.081005,-1.089883
Kuwait,-0.397505,-0.504872,-2.043912,-0.193466
...,...,...,...,...
Fiction,-0.996283,1.217438,3.030255,1.593772
The_Simpsons,0.802556,NaN,3.148226,0.988206
Sport,-0.963159,0.649759,3.551640,2.288856


In [ ]:
# sort by detour ratio (ratio of the number of dead ends an article has (difference between full path list content and simplified path list content))
composite_df.sort_values(by='detour_ratio')

,weighted_avg,avg_speed,unfinished_ratio,detour_ratio
article,,,,
Welding,1.771787,NaN,-0.542058,-1.312944
Coin,0.816901,-0.373638,-1.559253,-1.312944
List_of_countries,0.528985,-0.741666,-2.096106,-1.312944
United_States_Congress,-0.491675,NaN,0.521052,-1.312944
"Detroit,_Michigan",0.848160,3.627169,-0.748606,-1.312944
...,...,...,...,...
Yellowstone_National_Park,-0.989639,1.589729,2.118855,3.289356
Eukaryote,-1.550714,0.361369,2.631404,3.433178
Gas,-2.539718,-0.609033,0.473747,3.601376


## What now?

It would be cool to **get a composite score that incorporates all the metrics but what weight do we give each individual metric...?**

We can also seperate the game into two main objectives:
- **Reach your target in the least possible amount of clicks**.
    consider the following metrics
    - weighted_avg score
    - detour ratio 
    - maybe unfinished ratio
- **Reach your target as fast as possible**
    only really interested in time metric

Then we can test what article attributes correlate the most with high scores. And if they are similar for clicks and time.

Note that these metrics are more focused on article 'quality'. what I mean by that is that it is not the most important articles in the 'Network' (i.e. those that are the most used by players) that will have the highest scores.
     